# Streaming Consumer for Measuring Latency

In [1]:
from pykafka import KafkaClient
import numpy as np
#import msgpack
#import msgpack_numpy as m
#m.patch()
import time
import datetime
import dateutil.parser


zkKafka='c251-124.wrangler.tacc.utexas.edu:2181'
client = KafkaClient(zookeeper_hosts=zkKafka)
#client = KafkaClient(hosts='c251-142.wrangler.tacc.utexas.edu:9092')
topic = client.topics['latency']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()

NoBrokersAvailableError: Unable to connect to a broker to fetch metadata. See logs.

In [21]:
run_timestamp=datetime.datetime.now()
RESULT_FILE= "results/kafka-latency-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
try:
    os.makedirs("results")
except:
    pass
output_file=open(RESULT_FILE, "w")

In [22]:
restart_consumer = topic.get_simple_consumer(reset_offset_on_start=True)
message = restart_consumer.consume(block=True)    

In [24]:
message?

In [6]:
while True:
    message = consumer.consume(block=True)
    #print message.value
    now = time.time()
    #sent_time=datetime.datetime.strptime(message.value, "%Y-%m-%dT%H:%M:%S.%fZ")
    sent_time_string = message.value.split(";")[0]
    sleep_time =float(message.value.split(";")[1])
    sent_time = dateutil.parser.parse(sent_time_string)
    sent_time_ts = time.mktime(sent_time.timetuple())
    lat = now-sent_time_ts    
    result = "kafka, latency, %.5f, %.5f\n"%(1/sleep_time, lat)
    print(result)
    output_file.write(result)
    output_file.flush()

kafka, latency, 1.00000, 9.63784

kafka, latency, 1.00000, 4.64399

kafka, latency, 1.00000, 2.64606

kafka, latency, 1.00000, 1.64812

kafka, latency, 1.00000, 0.64989

kafka, latency, 1.00000, 0.26833

kafka, latency, 1.00000, 0.43718

kafka, latency, 1.00000, 0.55482

kafka, latency, 1.00000, 0.72299

kafka, latency, 1.00000, 0.84102

kafka, latency, 1.00000, 0.96125

kafka, latency, 1.00000, 0.13157

kafka, latency, 1.00000, 0.24994

kafka, latency, 1.00000, 0.41860

kafka, latency, 1.00000, 0.58694

kafka, latency, 1.00000, 0.70439

kafka, latency, 1.00000, 0.72527

kafka, latency, 1.00000, 0.76477

kafka, latency, 1.00000, 0.83366

kafka, latency, 1.00000, 0.84645

kafka, latency, 1.00000, 1.01657

kafka, latency, 10.00000, 1.02848

kafka, latency, 10.00000, 0.24659

kafka, latency, 10.00000, 0.46459

kafka, latency, 10.00000, 0.68291

kafka, latency, 10.00000, 0.90129

kafka, latency, 10.00000, 0.11912

kafka, latency, 10.00000, 0.33714

kafka, latency, 10.00000, 0.55538

kafka,

KeyboardInterrupt: 

In [19]:
topic.get_simple_consumer?